In [1]:
import pandas as pd
import numpy

#### Converting the positions to Attack, Midfield, Defence

In [3]:
Attackers = ["Left Winger", "Centre-Forward", "Right Winger", "Second Striker"]
Midfielders = ['Attacking Midfield', 'Central Midfield','Left Midfield', 'Defensive Midfield','Right Midfield']
Defenders = ['Left-Back','Right-Back', 'Centre-Back', 'Goalkeeper']

In [4]:
df.loc[df["Position"].isin(Attackers) , "Position"] = "Attacker"
df.loc[df["Position"].isin(Midfielders) , "Position"] = "Midfielder"
df.loc[df["Position"].isin(Defenders) , "Position"] = "Defender"

In [5]:
df = df.loc[df["Club"].isna() == False]

#### Adding Player Agents

In [6]:
from bs4 import BeautifulSoup
import requests

In [7]:
headers = {
    # 'Host' : 'https://www.zim.com/',
    'Connection': 'keep-alive',
    'User-Agent': 'Chrome/102.0.5005.63 Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.5 (KHTML, like Gecko) Safari/536.5',
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate',
    'Content-Type': 'text',
    'Accept-Language': 'en-US,en;q=0.8'
}

def get_data(URL):
    soup = BeautifulSoup(requests.get(URL, headers=headers).content, 'lxml')
    agent = 'Unknown'
    try:
        agent = soup.find('span', {'onclick': 'tmEvent("spielerprofil", "click", "berater-spielerdaten")'}).text.replace('\n', '')
    except:
        pass
    
    try:
        agent = soup.find('a', {'onclick': 'tmEvent("spielerprofil", "click", "berater-spielerdaten")'}).text.replace('\n', '')
    except:
        pass
    
    return agent

In [8]:
#Witsel Player link not working despite being correct
#df.iloc[270:285,:]

In [9]:
#df = df.drop(df.index[292])

In [10]:
agent = []
n = 0
for url in df["Player Link"] : 
    print(n)
    agent.append(get_data(url))
    n += 1        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
df.insert(3, "Agent", agent)

In [13]:
df.to_csv('Agents_Data.csv', encoding='utf-8')

#### Data for Outlet Analysis

In [14]:
df_outlet = pd.read_csv('Agents_Data.csv', index_col = [0] )
df_outlet = df_outlet.drop(['Rumour', 'Year', 'Player', 'Club', 'Interested_clubs', 'Clubs', 'Player Link'], axis=1)

In [15]:
df_outlet = df_outlet.iloc[:,[0,1,8]]

In [16]:
#df_outlet["Outlet"].value_counts()[0:12]

In [17]:
df_outlet = df_outlet.groupby('Outlet').filter(lambda x : len(x)>50)

In [18]:
df_outlet.to_csv('Outlet_Analysis_Data.csv', encoding='utf-8')

#### Data for Regression

In [71]:
df_reg = pd.read_csv('Agents_Data.csv', index_col = [0] )

In [72]:
df_reg = df_reg.groupby('Agent').filter(lambda x : len(x)>9)
df_reg = df_reg.groupby('League').filter(lambda x : len(x)>9)
df_reg = df_reg.groupby('Country').filter(lambda x : len(x)>9)
df_reg = df_reg.groupby('Outlet').filter(lambda x : len(x)>9)

In [73]:
#Storing for later
continuous = df_reg[["Market Value", "Join", "Age"]]

In [74]:
df_reg = df_reg.iloc[:,[2,3,6,8,10,15]]
#df_reg = df_reg.drop(['Rumour', 'Year', 'Player', 'Club', 'Interested_clubs', "Market Value", "Join", "Age"], axis=1)

In [76]:
features = pd.get_dummies(df_reg)

In [79]:
df_reg = pd.concat([continuous, features], axis=1)

In [81]:
df_reg.to_csv('Regression_Data.csv', encoding='utf-8')

#### Making sure we have categorical variables which make sense

In [26]:
df_analysis = pd.read_csv('Agents_Data.csv', index_col = [0] )
df_analysis = df_analysis.drop(['Rumour', 'Year', 'Player', 'Club', 'Interested_clubs', 'Clubs', 'Player Link'], axis=1)

In [27]:
df_len = len(df_analysis)

In [28]:
loss_Outlet  = df_len - len(df_analysis.groupby('Agent').filter(lambda x :   len(x) > 7))
loss_League  = df_len - len(df_analysis.groupby('League').filter(lambda x :  len(x) > 7))
loss_Country = df_len - len(df_analysis.groupby('Country').filter(lambda x : len(x) > 7))
loss_Agent   = df_len - len(df_analysis.groupby('Outlet').filter(lambda x :  len(x) > 7))

In [29]:
df_analysis = df_analysis.groupby('Agent').filter(lambda x : len(x)>9)
df_analysis = df_analysis.groupby('League').filter(lambda x : len(x)>9)
df_analysis = df_analysis.groupby('Country').filter(lambda x : len(x)>9)
df_analysis = df_analysis.groupby('Outlet').filter(lambda x : len(x)>9)

In [30]:
df_analysis

,Outlet,Agent,League,Position,Age,Country,Join,Market Value,Outcome
0,Fabrizio Romano,World Soccer Agency,Premier League,Attacker,21.0,Italy,1062.0,35.0,False
1,Mundo Deportivo,ESN,LaLiga,Defender,22.0,France,1094.0,60.0,True
9,Football Insider,Unique Sports Group,Ligue 1,Defender,23.0,France,1081.0,8.0,False
13,Independent Journalists,Unknown,Ligue 1,Attacker,24.0,Brazil,1033.0,30.0,False
19,Marca,CAA Stellar,LaLiga,Midfielder,26.0,Spain,3287.0,40.0,False
...,...,...,...,...,...,...,...,...,...
896,Telegraph,CAA Base Ltd,Liga Portugal,Defender,22.0,Spain,0.0,25.0,False
897,Fabrizio Romano,Goal Management,Premier League,Defender,27.0,Spain,2922.0,20.0,True
898,The Athletic,CAA Stellar,Premier League,Attacker,20.0,Sweden,181.0,20.0,False
902,Mail,Gestifute,Premier League,Defender,28.0,Portugal,1059.0,65.0,False


In [31]:
df_analysis.to_csv('Analysis_Data.csv', encoding='utf-8')

In [32]:
df_reg

,Outlet,Agent,League,Position,Age,Country,Join,Market Value,Outcome,Age,...,Country_Nigeria,Country_Portugal,Country_Scotland,Country_Serbia,Country_Spain,Country_Sweden,Country_Switzerland,Country_United States,Country_Uruguay,Country_Wales
0,Fabrizio Romano,World Soccer Agency,Premier League,Attacker,21.0,Italy,1062.0,35.0,False,21.0,...,0,0,0,0,0,0,0,0,0,0
1,Mundo Deportivo,ESN,LaLiga,Defender,22.0,France,1094.0,60.0,True,22.0,...,0,0,0,0,0,0,0,0,0,0
9,Football Insider,Unique Sports Group,Ligue 1,Defender,23.0,France,1081.0,8.0,False,23.0,...,0,0,0,0,0,0,0,0,0,0
13,Independent Journalists,Unknown,Ligue 1,Attacker,24.0,Brazil,1033.0,30.0,False,24.0,...,0,0,0,0,0,0,0,0,0,0
19,Marca,CAA Stellar,LaLiga,Midfielder,26.0,Spain,3287.0,40.0,False,26.0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,Telegraph,CAA Base Ltd,Liga Portugal,Defender,22.0,Spain,0.0,25.0,False,22.0,...,0,0,0,0,1,0,0,0,0,0
897,Fabrizio Romano,Goal Management,Premier League,Defender,27.0,Spain,2922.0,20.0,True,27.0,...,0,0,0,0,1,0,0,0,0,0
898,The Athletic,CAA Stellar,Premier League,Attacker,20.0,Sweden,181.0,20.0,False,20.0,...,0,0,0,0,0,1,0,0,0,0
902,Mail,Gestifute,Premier League,Defender,28.0,Portugal,1059.0,65.0,False,28.0,...,0,1,0,0,0,0,0,0,0,0
